# What’s up with `element.removeAttribute(attname)`

The following example runs to completion and produces the desired output unless I uncomment the two commented lines to try to remove the `@start` attribute with `node.removeAttribute('start)`. Hmm ...

In [8]:
from xml.dom.pulldom import CHARACTERS, START_ELEMENT, parseString, END_ELEMENT
from xml.dom.minidom import Document


class Stack(list):
    def push(self, item):
        self.append(item)

    def peek(self):
        return self[-1]


open_elements = Stack()
d = Document()
open_elements.push(d)

input = '''<root>
    <p start="p1"/>This is a <word start="w1"/>paragraph<word end="w1"/> that contains
    some stuff.<p end="p1"/>
</root>'''

for event, node in parseString(input):
    if event == START_ELEMENT and not node.hasAttribute('end'): # process pseudo-end-tags on END_ELEMENT event
        open_elements.peek().appendChild(node)
        open_elements.push(node)
#         if node.hasAttribute('start'):
#             node.removeAttribute('start')
    elif event == END_ELEMENT and not node.hasAttribute('start'): # process pseudo-start-tags on START_ELEMENT event
        open_elements.pop()
    elif event == CHARACTERS:
        t = d.createTextNode(node.data)
        open_elements.peek().appendChild(t)
    else:
        continue

print(open_elements[0].toxml())

<?xml version="1.0" ?><root>
    <p start="p1">This is a <word start="w1">paragraph</word> that contains
    some stuff.</p>
</root>


## How about cloning the node and removing the attribute from the clone?

In [17]:
from xml.dom.pulldom import CHARACTERS, START_ELEMENT, parseString, END_ELEMENT
from xml.dom.minidom import Document


class Stack(list):
    def push(self, item):
        self.append(item)

    def peek(self):
        return self[-1]


open_elements = Stack()
d = Document()
open_elements.push(d)

input = '''<root>
    <p start="p1"/>This is a <word start="w1"/>paragraph<word end="w1"/> that contains
    some stuff.<p end="p1"/>
</root>'''

for event, node in parseString(input):
    if event == START_ELEMENT and not node.hasAttribute('end'): # process pseudo-end-tags on END_ELEMENT event
        clone = node.cloneNode(deep=False)
        if clone.hasAttribute('start'):
            clone.removeAttribute('start')
        print(clone.toxml())
        open_elements.peek().appendChild(node)
        open_elements.push(node)
    elif event == END_ELEMENT and not node.hasAttribute('start'): # process pseudo-start-tags on START_ELEMENT event
        open_elements.pop()
    elif event == CHARACTERS:
        t = d.createTextNode(node.data)
        open_elements.peek().appendChild(t)
    else:
        continue

print(open_elements[0].toxml())

<root/>
<p/>
<word/>
<?xml version="1.0" ?><root>
    <p start="p1">This is a <word start="w1">paragraph</word> that contains
    some stuff.</p>
</root>
